# ModelTuning.ipynb

### This notebook contains a K-Fold Cross Validation algorithm to test various parameters for the supervised machine learning model used to classify Tweets in categories.

Author: Erik Puijk <br>
Date  : March 28, 2022

In [1]:
pip install -U scikit-learn

Requirement already up-to-date: scikit-learn in /home/erik/anaconda3/lib/python3.8/site-packages (1.0.2)
Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import json, csv, math, random
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score, classification_report
from sklearn import svm
import pyperclip as pc

In [2]:
Encoder = LabelEncoder()

In [3]:
def read_tweets(path):
    """ Read the Tweets from a given text file and return in JSON-format. """
    
    content = ""
    
    try:
        with open(path, 'r') as f:
            content = json.loads(f.read())
    except IOError:
        print("I/O error")
        
    return content

In [4]:
def stats_count(cat):
    """ Count the occurrences of each category in a list of categories. """
    
    occurrences = [[x,cat.count(x)] for x in sorted(set(cat))]
        
    for occ in occurrences:
        print('%4s: %4s | %3s%%' % (occ[0], occ[1], round(occ[1]/len(cat)*100)))
        
    print('')

In [5]:
def calc_avg_scores(scores, cats, k):
    """ Calculate an average score given a list of scores. """
    
    scores_avg = []
    
    # If a list of lists of scores is given, calculate the average per label
    if isinstance(scores[0], list): 
        
        for i in range(len(cats)):

            total = 0

            for j in range(k):
                total += scores[j][i]

            scores_avg.append(total / k)
            
    # If a list of average (f1) scores is given, just return that list
    else:
        return scores

    return scores_avg

In [6]:
def keysort(x):
    """ Calculate the average F1 score from a list of F1 scores, which is used to sort the scores. """
    
    return sum(x[1]) / len(x[1])

In [7]:
def ml(X_train, y_train, X_test, v_type, alg, ngram_min, ngram_max, min_df, max_df, max_features):
    """ Vectorize training and test set according to specific vectorizer and fit and predict using a specific
        SVM. 
        
        Tested vectorizers:
        - CountVectorizer (sklearn)
        - TfidfVectorizer (sklearn)
        
        Tested SVMS:
        - SVC (sklearn)
        - LinearSVC (sklearn)"""
    
    # Select appropriate vectorizer
    if v_type == 'count':
        vectorizer = CountVectorizer(analyzer='word', ngram_range=(ngram_min, ngram_max), min_df=min_df, \
                                     max_df=max_df, max_features=max_features)
    else:
        vectorizer = TfidfVectorizer(ngram_range=(ngram_min, ngram_max), min_df=min_df, max_df=max_df, \
                                     max_features=max_features, sublinear_tf=True)

    # Fit and transform tokens
    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Create and fit SVM
    if alg == 'svc':
        clf = svm.SVC(kernel='linear', C=0.6, class_weight='balanced')
    else:
        clf = svm.LinearSVC(C=0.5, class_weight='balanced', random_state=0)

    # Predict labels and return prediction
    clf.fit(X_train, y_train)
    
    return clf.predict(X_test)

In [8]:
def cross_val(v_type, X, y, ngram_min, ngram_max, min_df, max_df, max_features, cats, alg):
    """ Run 3-fold cross validation on the training set to create a training set and a validation set used
        to test different vectorization and modelling parameters. """
    
    scores = []
    k = 3
        
    # Run cross-validation and calculate scores
    kf = KFold(n_splits=k)
    for train, valid in kf.split(X, y):
        X_train_t = [X[i] for i in train]
        y_train_t = [y[i] for i in train]
        X_valid_t = [X[i] for i in valid]
        y_valid_t = [y[i] for i in valid]
        
        pred = ml(X_train_t, y_train_t, X_valid_t, v_type, alg, ngram_min, ngram_max, min_df, max_df, max_features)
        
        scores.append(f1_score(y_valid_t, pred, average='micro', zero_division=0))
    
    # Return the configuration of the model together with the average of scores
    return [[ngram_min, ngram_max, min_df, max_df, max_features], calc_avg_scores(scores, cats, k)]

In [12]:
def test_configurations(v_type, test_validation, tweets, labels, alg, rs):
    """ Test different configurations for different models and compare the micro-f1 scores to select the best
        model. """
    
    cats = list(set(labels))
    
    # Define most promosing model parameters for vectorization according to categorization type
    if len(cats) == 3:
        # Content
        ngrams = [1, 2]
        min_dfs = [1]
        max_dfs = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
        max_features = [3000]
    else:
        # Activation
        ngrams = [1, 2]
        min_dfs = [1]
        max_dfs = [0.1, 0.2, 0.4, 0.6, 0.8, 1.0]
        max_features = [450, 500, 1000, 2000, 3000, None]
    
    # Split data set into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(tweets, labels, test_size=0.3, random_state=rs)
        
    #stats_count(list(y_train))

    # Encode labels
    y_train = Encoder.fit_transform(y_train)
    y_test = Encoder.fit_transform(y_test)
    
    scores = []
    i = 0
    i_max = (len(ngrams)**2 - math.factorial(len(ngrams)-1)) * len(min_dfs) * len(max_dfs) * len(max_features)
    
    # Combine each parameter and run the model with cross validation
    for ngram_min in ngrams:
        for ngram_max in ngrams:
            if ngram_min > ngram_max:
                continue
            for min_df in min_dfs:
                for max_df in max_dfs:
                    for max_feature in max_features:
                        i += 1
                        
                        # Call the function for the model and append the score
                        scores.append(cross_val(v_type, X_train, y_train, ngram_min, ngram_max, min_df, \
                                                max_df, max_feature, cats, alg))
    
    # Sort the scores descending and print them
    scores.sort(key=keysort, reverse=True)
    
    prnt = ''
    
    if test_validation:
        # Select the best configurations
        best_score = keysort(scores[0])
        best_configs = []
        for score in scores:
            if keysort(score) == best_score:
                best_configs.append(score)

        # Use the best configurations to validate model on the test set
        for config, score in best_configs:
            prnt += (str([config, score]) + '\n')
            
            pred = ml(X_train, y_train, X_test, v_type, alg, config[0], config[1], config[2], config[3], config[4])

            prnt += (classification_report(y_test, pred, \
                                           target_names=Encoder.inverse_transform(list(set(y_test))), \
                                           zero_division=0))

    print(prnt)

In [15]:
# Read all Tweets
tweets = read_tweets('source/tweets_all_preprocessed_exc_stopwords.txt')

# Select gold-standard (labeled) Tweets
tweets_gs = [tweet for tweet in tweets if tweet['memo'] == 'gold_standard']

# Select the text from those Tweets
tweets_text = [tweet['text'] for tweet in tweets_gs]

# Select the labels from those Tweets
labels_con = [tweet['cat_con'] for tweet in tweets_gs]
labels_act = [tweet['cat_act'] for tweet in tweets_gs]

# Define number of different random states to test parameters on
i_max = 10

# For each random state, run the algorithm and see which parameter settings get the best results using K-fold
# cross validation
for i in range(0, i_max):
    print('RS=%s:' % (i))
    #test_configurations('count', True, tweets_text, labels_con, 'linear_svc', i)
    test_configurations('tfidf', True, tweets_text, labels_act, 'svc', i)

RS=0:
[[1, 1, 1, 0.1, 450], [0.8256880733944955, 0.779816513761468, 0.7407407407407407]]
              precision    recall  f1-score   support

         CON       1.00      0.67      0.80         6
         FOL       0.78      0.67      0.72        27
        NONE       0.88      0.92      0.90        93
         SUP       0.67      0.71      0.69        14

    accuracy                           0.84       140
   macro avg       0.83      0.74      0.78       140
weighted avg       0.84      0.84      0.84       140

RS=1:
[[1, 1, 1, 0.1, 450], [0.7889908256880734, 0.81651376146789, 0.8148148148148148]]
              precision    recall  f1-score   support

         CON       0.62      1.00      0.77         5
         FOL       0.77      0.53      0.63        32
        NONE       0.81      0.90      0.85        81
         SUP       0.75      0.68      0.71        22

    accuracy                           0.79       140
   macro avg       0.74      0.78      0.74       140
weighted

KeyboardInterrupt: 